In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mahotas
import cv2
import os
import h5py
images_per_class       = 800
fixed_size             = tuple((256, 256))
train_path             = "dataset/train"
h5_train_data          = 'output/train_data.h5'
h5_train_labels        = 'output/train_labels.h5'
bins                   = 8

In [2]:
def rgb_bgr(image):
    rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return rgb_img

In [3]:
def bgr_hsv(rgb_img):
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)
    return hsv_img

In [4]:
def img_segmentation(rgb_img,hsv_img):
    lower_green = np.array([25,0,20])
    upper_green = np.array([100,255,255])
    healthy_mask = cv2.inRange(hsv_img, lower_green, upper_green)
    result = cv2.bitwise_and(rgb_img,rgb_img, mask=healthy_mask)
    lower_brown = np.array([10,0,10])
    upper_brown = np.array([30,255,255])
    disease_mask = cv2.inRange(hsv_img, lower_brown, upper_brown)
    disease_result = cv2.bitwise_and(rgb_img, rgb_img, mask=disease_mask)
    final_mask = healthy_mask + disease_mask
    final_result = cv2.bitwise_and(rgb_img, rgb_img, mask=final_mask)
    return final_result


In [5]:
train_labels = os.listdir(train_path)
train_labels.sort()
print(train_labels)
Hog_features = []
labels = []


['diseased', 'healthy']


In [6]:
from skimage.feature import hog
from skimage import exposure
for training_name in train_labels:
    dir = os.path.join(train_path, training_name)
    current_label = training_name
    for x in range(1, images_per_class + 1):
        file = os.path.join(dir, str(x) + ".jpg")
        image = cv2.imread(file)
        image = cv2.resize(image, fixed_size)
        RGB_BGR = rgb_bgr(image)
        BGR_HSV = bgr_hsv(RGB_BGR)
        IMG_SEGMENT = img_segmentation(RGB_BGR, BGR_HSV) 
        gray_image = cv2.cvtColor(IMG_SEGMENT, cv2.COLOR_BGR2GRAY)
        fd_hog = hog(gray_image, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
        labels.append(current_label)
        Hog_features.append(fd_hog)
    
    print("[STATUS] processed folder: {}".format(current_label))
    
print("[STATUS] completed Hog Feature Extraction...")

[STATUS] processed folder: diseased
[STATUS] processed folder: healthy
[STATUS] completed Hog Feature Extraction...


In [7]:
import os

In [8]:
print("[STATUS] feature vector size {}".format(np.array(Hog_features).shape))

[STATUS] feature vector size (1600, 34596)


In [9]:
targetNames = np.unique(labels)
le          = LabelEncoder()
target      = le.fit_transform(labels)
print("[STATUS] training labels encoded...")

[STATUS] training labels encoded...


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler            = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(Hog_features)
print("[STATUS] feature vector normalized...")

[STATUS] feature vector normalized...


In [11]:
rescaled_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
rescaled_features.shape

(1600, 34596)

In [13]:
num_trees = 100
test_size = 0.20
seed      = 9

In [14]:
from sklearn.model_selection import train_test_split
(trainDataHog, testDataHog, trainLabelsHog, testLabelsHog) = train_test_split(np.array(rescaled_features), np.array(target),test_size=test_size,random_state=seed)
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataHog.shape))
print("Test data   : {}".format(testDataHog.shape)) 

[STATUS] splitted train and test data...
Train data  : (1280, 34596)
Test data   : (320, 34596)


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

#clf2  = RandomForestClassifier(n_estimators=num_trees, random_state=seed)
#clf2 = KNeighborsClassifier(n_neighbors=5) 
#clf2 = DecisionTreeClassifier(random_state=seed)
#clf2 = GaussianNB()
clf2 = SVC(kernel='linear', probability=True)
#clf2 = LogisticRegression(random_state=seed,max_iter=1000) 
clf2.fit(trainDataHog, trainLabelsHog)
y_predict_Hog = clf2.predict_proba(testDataHog)

In [16]:
y_predict_Hog.shape

(320, 2)

In [17]:
y_predict_Hog

array([[8.67975173e-01, 1.32024827e-01],
       [2.54940002e-02, 9.74506000e-01],
       [1.33745396e-01, 8.66254604e-01],
       [9.15384286e-01, 8.46157144e-02],
       [9.24325302e-01, 7.56746978e-02],
       [2.29236950e-02, 9.77076305e-01],
       [7.57530982e-02, 9.24246902e-01],
       [5.80380504e-02, 9.41961950e-01],
       [1.76331904e-01, 8.23668096e-01],
       [7.47991376e-01, 2.52008624e-01],
       [5.83057054e-01, 4.16942946e-01],
       [2.02137326e-01, 7.97862674e-01],
       [8.73978907e-01, 1.26021093e-01],
       [6.61392245e-01, 3.38607755e-01],
       [9.17837590e-01, 8.21624096e-02],
       [2.32819808e-01, 7.67180192e-01],
       [9.10639448e-01, 8.93605520e-02],
       [5.12814281e-01, 4.87185719e-01],
       [1.63734447e-01, 8.36265553e-01],
       [2.04362978e-01, 7.95637022e-01],
       [6.92109102e-02, 9.30789090e-01],
       [1.42501695e-01, 8.57498305e-01],
       [5.00000000e-01, 5.00000000e-01],
       [2.12511831e-02, 9.78748817e-01],
       [9.524378

In [18]:
labels1 = []
Histogram_features=[]

In [19]:
# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [20]:
from skimage import exposure
for training_name in train_labels:
    dir = os.path.join(train_path, training_name)
    current_label = training_name
    for x in range(1, images_per_class + 1):
        file = os.path.join(dir, str(x) + ".jpg")
        image = cv2.imread(file)
        image = cv2.resize(image, fixed_size)
        RGB_BGR = rgb_bgr(image)
        BGR_HSV = bgr_hsv(RGB_BGR)
        IMG_SEGMENT = img_segmentation(RGB_BGR, BGR_HSV) 
        fv_Histogram= fd_histogram(IMG_SEGMENT)
        labels1.append(current_label)
        Histogram_features.append(fv_Histogram)
    
    print("[STATUS] processed folder: {}".format(current_label))
    
print("[STATUS] completed Histogram Feature Extraction...")

[STATUS] processed folder: diseased
[STATUS] processed folder: healthy
[STATUS] completed Histogram Feature Extraction...


In [21]:
print("[STATUS] feature vector size {}".format(np.array(Histogram_features).shape))

[STATUS] feature vector size (1600, 512)


In [22]:
print("[STATUS] training Labels {}".format(np.array(labels1).shape))

[STATUS] training Labels (1600,)


In [23]:
targetNames = np.unique(labels1)
le          = LabelEncoder()
target1      = le.fit_transform(labels1)
print("[STATUS] training labels encoded...")

[STATUS] training labels encoded...


In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler            = MinMaxScaler(feature_range=(0, 1))
rescaled_features1 = scaler.fit_transform(Histogram_features)
print("[STATUS] feature vector normalized...")

[STATUS] feature vector normalized...


In [25]:
rescaled_features1.shape

(1600, 512)

In [26]:
from sklearn.model_selection import train_test_split
(trainDataHistogram, testDataHistogram, trainLabelsHistogram, testLabelsHistogram) = train_test_split(np.array(rescaled_features1), np.array(target1),test_size=test_size,random_state=seed)
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataHistogram.shape))
print("Test data   : {}".format(testDataHistogram.shape)) 

[STATUS] splitted train and test data...
Train data  : (1280, 512)
Test data   : (320, 512)


In [27]:
rescaled_features1

array([[0.89868999, 0.03427632, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.97074117, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.96591603, 0.01332912, 0.02549925, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9815963 , 0.01239635, 0.03557221, ..., 0.        , 0.        ,
        0.        ],
       [0.96860402, 0.01473364, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.9613786 , 0.0379812 , 0.09687979, ..., 0.        , 0.        ,
        0.        ]])

In [28]:
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
#clf  = RandomForestClassifier(n_estimators=num_trees, random_state=seed)
#clf = LogisticRegression(random_state=seed,max_iter=1000) 
#clf = KNeighborsClassifier(n_neighbors=5) 
clf = DecisionTreeClassifier(random_state=seed)
#clf = GaussianNB()
#clf = SVC(kernel='linear', probability=True)

clf.fit(trainDataHistogram, trainLabelsHistogram)
y_predict_Histo = clf.predict_proba(testDataHistogram)


In [29]:
y_predict_Histo.shape

(320, 2)

In [30]:
y_predict_Histo

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [31]:
import numpy as np
final=np.multiply(y_predict_Hog ,y_predict_Histo)

In [32]:
final

array([[8.67975173e-01, 0.00000000e+00],
       [0.00000000e+00, 9.74506000e-01],
       [0.00000000e+00, 8.66254604e-01],
       [9.15384286e-01, 0.00000000e+00],
       [9.24325302e-01, 0.00000000e+00],
       [0.00000000e+00, 9.77076305e-01],
       [0.00000000e+00, 9.24246902e-01],
       [0.00000000e+00, 9.41961950e-01],
       [1.76331904e-01, 0.00000000e+00],
       [0.00000000e+00, 2.52008624e-01],
       [0.00000000e+00, 4.16942946e-01],
       [0.00000000e+00, 7.97862674e-01],
       [8.73978907e-01, 0.00000000e+00],
       [6.61392245e-01, 0.00000000e+00],
       [9.17837590e-01, 0.00000000e+00],
       [0.00000000e+00, 7.67180192e-01],
       [0.00000000e+00, 8.93605520e-02],
       [0.00000000e+00, 4.87185719e-01],
       [1.63734447e-01, 0.00000000e+00],
       [0.00000000e+00, 7.95637022e-01],
       [0.00000000e+00, 9.30789090e-01],
       [0.00000000e+00, 8.57498305e-01],
       [5.00000000e-01, 0.00000000e+00],
       [0.00000000e+00, 9.78748817e-01],
       [9.524378

In [33]:
final.shape

(320, 2)

In [34]:
import numpy as np

def maxelement_efficient(arr):
  max_element_per_row = np.amax(arr, axis=1)
  return max_element_per_row

y_predictfinal = maxelement_efficient(final)

print(y_predictfinal)


[8.67975173e-01 9.74506000e-01 8.66254604e-01 9.15384286e-01
 9.24325302e-01 9.77076305e-01 9.24246902e-01 9.41961950e-01
 1.76331904e-01 2.52008624e-01 4.16942946e-01 7.97862674e-01
 8.73978907e-01 6.61392245e-01 9.17837590e-01 7.67180192e-01
 8.93605520e-02 4.87185719e-01 1.63734447e-01 7.95637022e-01
 9.30789090e-01 8.57498305e-01 5.00000000e-01 9.78748817e-01
 9.52437863e-01 8.08719372e-01 9.55012546e-01 1.41408254e-01
 8.38662876e-01 1.87202230e-02 9.24315187e-01 8.29380466e-01
 6.56009563e-01 6.84791748e-01 3.59677578e-01 9.93322585e-01
 8.16675007e-01 1.53152568e-01 7.92124223e-01 4.27761624e-01
 8.81207539e-01 7.25021508e-01 8.64620145e-01 2.49790759e-01
 6.82547846e-01 2.84222267e-01 5.94901840e-01 4.65892446e-01
 9.80841733e-01 9.01615533e-01 9.07826813e-01 9.31804434e-01
 7.86483007e-01 7.79546076e-01 4.64823791e-01 8.80646481e-01
 3.27422483e-01 9.09624841e-01 5.08389082e-01 9.53566779e-01
 1.20089363e-02 8.21990910e-01 6.00980838e-01 7.30898598e-01
 9.74389448e-01 9.982415

In [35]:
y_predictfinal=np.array(y_predictfinal)


In [36]:
y_predictfinal

array([8.67975173e-01, 9.74506000e-01, 8.66254604e-01, 9.15384286e-01,
       9.24325302e-01, 9.77076305e-01, 9.24246902e-01, 9.41961950e-01,
       1.76331904e-01, 2.52008624e-01, 4.16942946e-01, 7.97862674e-01,
       8.73978907e-01, 6.61392245e-01, 9.17837590e-01, 7.67180192e-01,
       8.93605520e-02, 4.87185719e-01, 1.63734447e-01, 7.95637022e-01,
       9.30789090e-01, 8.57498305e-01, 5.00000000e-01, 9.78748817e-01,
       9.52437863e-01, 8.08719372e-01, 9.55012546e-01, 1.41408254e-01,
       8.38662876e-01, 1.87202230e-02, 9.24315187e-01, 8.29380466e-01,
       6.56009563e-01, 6.84791748e-01, 3.59677578e-01, 9.93322585e-01,
       8.16675007e-01, 1.53152568e-01, 7.92124223e-01, 4.27761624e-01,
       8.81207539e-01, 7.25021508e-01, 8.64620145e-01, 2.49790759e-01,
       6.82547846e-01, 2.84222267e-01, 5.94901840e-01, 4.65892446e-01,
       9.80841733e-01, 9.01615533e-01, 9.07826813e-01, 9.31804434e-01,
       7.86483007e-01, 7.79546076e-01, 4.64823791e-01, 8.80646481e-01,
      

In [37]:
y_predictfinal.shape

(320,)

In [38]:
import numpy as np

def maxelement_column(arr):
  max_element_column = np.argmax(arr, axis=1)
  return max_element_column

y_predictlabel = maxelement_column(final)
print(y_predictlabel)

[0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 0 0 0 1 1
 0 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 1 0 0
 0 1 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0
 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 1 0 0
 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1
 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0]


In [39]:
y_predictlabel=np.array(y_predictlabel)

In [40]:
y_predictlabel.shape

(320,)

In [41]:
y_predictlabel

array([0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,

In [42]:
from sklearn.metrics import accuracy_score
result= accuracy_score(testLabelsHistogram,y_predictlabel)

In [43]:
result

0.928125

In [158]:
Haralick_features = []
labels2       = []

In [159]:
def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

In [160]:
for training_name in train_labels:
    dir = os.path.join(train_path, training_name)
    current_label = training_name
    for x in range(1, images_per_class + 1):
        file = os.path.join(dir, str(x) + ".jpg")
        image = cv2.imread(file)
        image = cv2.resize(image, fixed_size)
        RGB_BGR = rgb_bgr(image)
        BGR_HSV = bgr_hsv(RGB_BGR)
        IMG_SEGMENT = img_segmentation(RGB_BGR, BGR_HSV) 
        fv_Haralick = fd_haralick(IMG_SEGMENT)
        labels2.append(current_label)
        Haralick_features.append(fv_Haralick)
    
    print("[STATUS] processed folder: {}".format(current_label))
    
print("[STATUS] completed Haralick Feature Extraction...")

[STATUS] processed folder: diseased
[STATUS] processed folder: healthy
[STATUS] completed Haralick Feature Extraction...


In [161]:
print("[STATUS] feature vector size {}".format(np.array(Haralick_features).shape))

[STATUS] feature vector size (1600, 13)


In [162]:
print("[STATUS] training Labels {}".format(np.array(labels2).shape))

[STATUS] training Labels (1600,)


In [163]:
targetNames = np.unique(labels2)
le          = LabelEncoder()
target2     = le.fit_transform(labels2)
print("[STATUS] training labels encoded...")

[STATUS] training labels encoded...


In [164]:
from sklearn.preprocessing import MinMaxScaler
scaler            = MinMaxScaler(feature_range=(0, 1))
rescaled_features2 = scaler.fit_transform(Haralick_features)
print("[STATUS] feature vector normalized...")

[STATUS] feature vector normalized...


In [165]:
print("[STATUS] target labels: {}".format(target2))
print("[STATUS] target labels shape: {}".format(target2.shape))

[STATUS] target labels: [0 0 0 ... 1 1 1]
[STATUS] target labels shape: (1600,)


In [166]:
from sklearn.model_selection import train_test_split
(trainDataHaralick, testDataHaralick, trainLabelsHaralick, testLabelsHaralick) = train_test_split(np.array(rescaled_features2), np.array(target2),test_size=test_size,random_state=seed)
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataHaralick.shape))
print("Test data   : {}".format(testDataHaralick.shape)) 

[STATUS] splitted train and test data...
Train data  : (1280, 13)
Test data   : (320, 13)


In [167]:
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

clf  = RandomForestClassifier(n_estimators=num_trees, random_state=seed)
#clf = LogisticRegression(random_state=seed,max_iter=1000) 
#clf = KNeighborsClassifier(n_neighbors=5) 
#clf = DecisionTreeClassifier(random_state=seed)
#clf = GaussianNB()
#clf = SVC(kernel='linear', probability=True)

clf.fit(trainDataHaralick, trainLabelsHaralick)
y_predict_Hara = clf.predict_proba(testDataHaralick)

In [168]:
y_predict_Hara

array([[0.44, 0.56],
       [0.12, 0.88],
       [0.7 , 0.3 ],
       [0.81, 0.19],
       [0.94, 0.06],
       [0.12, 0.88],
       [0.04, 0.96],
       [0.28, 0.72],
       [0.81, 0.19],
       [0.44, 0.56],
       [0.43, 0.57],
       [0.19, 0.81],
       [0.87, 0.13],
       [0.69, 0.31],
       [0.12, 0.88],
       [0.  , 1.  ],
       [0.54, 0.46],
       [0.03, 0.97],
       [0.72, 0.28],
       [0.25, 0.75],
       [0.09, 0.91],
       [0.18, 0.82],
       [0.4 , 0.6 ],
       [0.02, 0.98],
       [0.75, 0.25],
       [0.67, 0.33],
       [0.72, 0.28],
       [0.38, 0.62],
       [0.99, 0.01],
       [0.71, 0.29],
       [0.36, 0.64],
       [0.22, 0.78],
       [0.59, 0.41],
       [0.68, 0.32],
       [0.39, 0.61],
       [0.91, 0.09],
       [0.15, 0.85],
       [0.53, 0.47],
       [0.25, 0.75],
       [0.69, 0.31],
       [0.67, 0.33],
       [0.86, 0.14],
       [0.75, 0.25],
       [0.61, 0.39],
       [0.61, 0.39],
       [0.25, 0.75],
       [0.79, 0.21],
       [0.54,

In [169]:
import numpy as np
res=np.multiply(y_predict_Hara , final)

In [170]:
res.shape

(320, 2)

In [171]:
import numpy as np

def maxelement_efficient(arr):
  max_element_per_row = np.amax(arr, axis=1)
  return max_element_per_row

y_predictfinal2 = maxelement_efficient(res)

print(y_predictfinal2)


[3.81909076e-01 8.57565280e-01 2.59876381e-01 7.41461271e-01
 8.68865784e-01 8.59827148e-01 8.87277026e-01 6.78212604e-01
 1.42828843e-01 1.41124830e-01 2.37657479e-01 6.46268766e-01
 7.60361649e-01 4.56360649e-01 1.10140511e-01 7.67180192e-01
 4.11058539e-02 4.72570148e-01 1.17888802e-01 5.96727766e-01
 8.47018072e-01 7.03148610e-01 2.00000000e-01 9.59173841e-01
 7.14328397e-01 5.41841979e-01 6.87609033e-01 8.76731177e-02
 8.30276247e-01 5.42886467e-03 5.91561720e-01 6.46916764e-01
 3.87045642e-01 4.65658389e-01 2.19403323e-01 9.03923552e-01
 6.94173756e-01 8.11708610e-02 5.94093168e-01 2.95155520e-01
 5.90409051e-01 6.23518497e-01 6.48465109e-01 1.52372363e-01
 2.66193660e-01 7.10555667e-02 4.69972454e-01 2.51581921e-01
 9.71033316e-01 8.11453979e-01 8.26122400e-01 6.89535281e-01
 7.47158857e-01 4.98909488e-01 4.36934364e-01 6.69291325e-01
 2.84857560e-01 4.00234930e-01 7.11744715e-02 8.48674433e-01
 3.00223408e-03 5.75393637e-01 5.40882754e-01 7.08971640e-01
 5.55401986e-01 8.584877

In [172]:
y_predictfinal=np.array(y_predictfinal2)

In [173]:
import numpy as np

def maxelement_column(arr):
  max_element_column = np.argmax(arr, axis=1)
  return max_element_column

y_predictlabel2 = maxelement_column(res)
print(y_predictlabel2)

[0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 0 0 0 1 1
 0 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 1 0 0
 0 1 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0
 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 1 0 0
 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1
 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0]


In [174]:
y_predictlabel2.shape

(320,)

In [175]:
from sklearn.metrics import accuracy_score
result2= accuracy_score(testLabelsHistogram,y_predictlabel2)

In [176]:
result2

0.928125